RNNs are designed to operate on a sequence of data.

- It has proven to be successful and effective on Natural Language Processing problems where the text sequence is provided as the input to the model.
- In this example, we will look at the most popular type of RNN, Long Short Term Memory (LSTM).
- It can be used in a model to accept a sequence of data as input and make a prediction such as assigning a class label or predicting numerical value like the next value or values in the sequence.
- Here, we will be using a car sales dataset to demonstrate an LSTM RNN for univariate time series forecasting.
- It involves predicting the number of car sales per month. 

In [28]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

In [3]:
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = pd.read_csv(path, header=0, index_col=0, squeeze=True)

In [4]:
df.head(20)

Month
1960-01     6550
1960-02     8728
1960-03    12026
1960-04    14395
1960-05    14587
1960-06    13791
1960-07     9498
1960-08     8251
1960-09     7049
1960-10     9545
1960-11     9364
1960-12     8456
1961-01     7237
1961-02     9374
1961-03    11837
1961-04    13784
1961-05    15926
1961-06    13821
1961-07    11143
1961-08     7975
Name: Sales, dtype: int64

In [21]:
#Splitting Sequence into Samples

#We will approach this problem by taking a window of the last five months of data to predict the current month's data.

#To achieve this, we will define a function split_sequence(), which will split the sequence of windows of data, 
#which is appropriate for fitting a supervised model like LSTM.

def split_sequence(sequence, n_steps):
  X, y = list(), list()
  for i in range(len(sequence)):
    end_ix = i + n_steps
    if end_ix > len(sequence)-1:
      break
    seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
    #print(seq_x)
    #print(seq_y)
    X.append(seq_x)
    y.append(seq_y)
  return np.asarray(X), np.asarray(y)

In [22]:
#convert the data type of the values of the DataFrame to float and specify the window size to split the sequence into samples.

values = df.values.astype('float32')
n_steps = 5

In [23]:

#Now, let us call the function to split the sequence into samples.

X, y = split_sequence(values, n_steps)

In [24]:
X[:10]

array([[ 6550.,  8728., 12026., 14395., 14587.],
       [ 8728., 12026., 14395., 14587., 13791.],
       [12026., 14395., 14587., 13791.,  9498.],
       [14395., 14587., 13791.,  9498.,  8251.],
       [14587., 13791.,  9498.,  8251.,  7049.],
       [13791.,  9498.,  8251.,  7049.,  9545.],
       [ 9498.,  8251.,  7049.,  9545.,  9364.],
       [ 8251.,  7049.,  9545.,  9364.,  8456.],
       [ 7049.,  9545.,  9364.,  8456.,  7237.],
       [ 9545.,  9364.,  8456.,  7237.,  9374.]], dtype=float32)

In [25]:
y[:10]

array([13791.,  9498.,  8251.,  7049.,  9545.,  9364.,  8456.,  7237.,
        9374., 11837.], dtype=float32)

In [17]:
X.shape,y.shape

((103, 5), (103,))

In [26]:

#Let us reshape the data X into samples, timesteps, and features.

X = X.reshape((X.shape[0], X.shape[1], 1))
print(X.shape)



(103, 5, 1)


In [29]:
#Let us split up the dataset as train and test datasets.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
#Here, the dataset is split up into the train and test datasets, where 77% of the data will be in the training dataset, and the remaining 33% will be the test dataset.

In [30]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((69, 5, 1), (34, 5, 1), (69,), (34,))

# Defining the Model

In [31]:
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps,1)))
model.add(Dense(50, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(1))

#Here, the model has been defined, and appropriate input and output layers are given. 
#The input layer will consist of n_steps from the parameter input_shape. 
#The first hidden layer has 100 nodes, which is an LSTM layer with the activation layer as relu, 
#and the next two layers are Dense layers, with both having 50 nodes and the activation layer as relu. 
#The output layer has one node.

# Compiling the Model

In [32]:
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Fitting the Model

In [33]:
model.fit(X_train, y_train, epochs=350, batch_size=32, verbose=2)

Epoch 1/350
3/3 - 2s - loss: 226043392.0000 - mae: 14423.2109
Epoch 2/350
3/3 - 0s - loss: 209542368.0000 - mae: 13825.4443
Epoch 3/350
3/3 - 0s - loss: 191797904.0000 - mae: 13230.5293
Epoch 4/350
3/3 - 0s - loss: 177354352.0000 - mae: 12676.0703
Epoch 5/350
3/3 - 0s - loss: 153113968.0000 - mae: 11707.4727
Epoch 6/350
3/3 - 0s - loss: 126181672.0000 - mae: 10495.3037
Epoch 7/350
3/3 - 0s - loss: 91689088.0000 - mae: 8790.9512
Epoch 8/350
3/3 - 0s - loss: 61878540.0000 - mae: 6788.0479
Epoch 9/350
3/3 - 0s - loss: 33637404.0000 - mae: 4666.8647
Epoch 10/350
3/3 - 0s - loss: 23151254.0000 - mae: 3911.8210
Epoch 11/350
3/3 - 0s - loss: 22422182.0000 - mae: 3726.7578
Epoch 12/350
3/3 - 0s - loss: 25730298.0000 - mae: 3974.4116
Epoch 13/350
3/3 - 0s - loss: 22988512.0000 - mae: 3878.4709
Epoch 14/350
3/3 - 0s - loss: 18835316.0000 - mae: 3534.7329
Epoch 15/350
3/3 - 0s - loss: 17547538.0000 - mae: 3274.3635
Epoch 16/350
3/3 - 0s - loss: 15654114.0000 - mae: 3164.6653
Epoch 17/350
3/3 - 0s

# Evaluating the Model

In [35]:
mse, mae = model.evaluate(X_test, y_test, verbose=2)
print('MSE: %.3f, RMSE: %.3f, MAE: %.3f' % (mse, np.sqrt(mse), mae))
#Here, the model will be evaluated on the test dataset, and the appropriate loss and accuracy will be saved in the variables mse and mae.

2/2 - 0s - loss: 10862082.0000 - mae: 2724.4128
MSE: 10862082.000, RMSE: 3295.767, MAE: 2724.413


# Making Predictions

In [37]:
#Let us now make a prediction on the new data to check the prediction values it gives. Let us predict for one row of data.

row = np.asarray([18024.0, 16722.0, 14385.0, 21342.0, 17180.0]).reshape((1, n_steps, 1))
yhat = model.predict(row)
print('Predicted: %.3f' % (yhat))
#Here, the values for all the input features are given in the variable row as an array, and we are making predictions to identify the value of the target variable.

Predicted: 18816.848
